# Visualising sequence quality in Jalview

In one of our undergraduate practicals we use PCR to prepare a DNA sample for sanger sequencing and then retrieve the results as AB1 and fasta files. The base calling takes no heed to the quality score in the sequence, with the result that the student must then navigate back and forth between trace file and alignment to assess the validity of variants.

In this note we take the AB1 files generated as the sequencer output and extract the called sequences as a fasta file, and the quality scores as a Jalview feature file.  These can then be visualised in Jalview.


In [4]:
from Bio import SeqIO
import os


The imports we use are BioPython (I have 1.69 installed with anaconda) for handling the sequence conversion and os for navigating the directory structure.

In the code below we enter the path to the directory of .ab1 files to process, and open a fasta and jalview feature file for output.

In addition we create an array to keep all our sequences.

In [1]:
datadir = 'ab1/ab1'
outfile = 'sequences.fasta'

features = 'sequences.jff'
ffh = open(features, 'w')
header='''phred score\tff0000|ffaa|0|95'''
print(header, file=ffh)

sequences = []

First define a method for processing an ab1 file. This will use Biopython to extract the phred score (sequencw quality) and add the sequence record to a list for later output.

In [12]:
def extract_info(file, featurefile):
    record = list(SeqIO.parse(file, 'abi'))[0]
    sequence = record.seq
    quality = record.letter_annotations['phred_quality']
    #set the id so that it appears in the fasta file correctly
    record.id = record.name
    sequences.append(record)
    #print out the phred score, one base per line.
    for i in range(len(sequence)):
        spos=i+1
        print('\t'.join([str(quality[i]), record.name,"-1", str(spos), str(spos), 
                         "phred score", str(quality[i])]),file=featurefile)
    
   
    

Now loop through all the files in datadir, processing any ab1 files.

In [13]:
for abifile in os.listdir(datadir):
    if not abifile.endswith('ab1'):
        continue
    extract_info(os.path.join(datadir, abifile), ffh)

ffh.close()


And write out the called sequences in FASTA format.

In [14]:
SeqIO.write(sequences, outfile, 'fasta')

95

Read the sequences into Jalview in the usual way. Then chose _File > Load Features/Annotations_ from the menu in the alignment window. This will load the quality scores and you should end up with something which looks like this:
![Alignment](Capture.PNG)
